In [1]:
#Libraries import
import cv2                     
import numpy as np
import matplotlib.pyplot as plt 
from scipy import ndimage
from itertools import chain, islice

In [30]:
#Image processing before indetification
colour_img = cv2.imread('./uno_images/b0.jpg')
rotated_img = ndimage.rotate(colour_img, 90)                              #rotate image by 90 degrees, increase user ease of use
bw_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)                    #convert to a black and white image

#Debbugging/Testing Binarisation
#img_sm = cv2.blur(img, (1, 1))                                           #not required for the images provided, no changes can be noted        
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_BINARY)    #accurate countours, seems to require more antialising
thr_value, th_img = cv2.threshold(bw_img,150, 400, cv2.THRESH_BINARY_INV) #accurate countours, smoother edges compared to regular binary
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_TRUNC)     #accurate canny that identifies depth, wrong contours
#thr_value, img_th = cv2.threshold(bw_img,150, 400, cv2.THRESH_TOZERO)    #accurate canny with noise, issues with contours
#thr_value, img_th = cv2.threshold(bw_img,50, 100, cv2.THRESH_TOZERO_INV) #inaccurate countours 

#Morphology correction (very small changes with images provided, helps with countour accuracy)
kernel = np.ones((3, 3), np.uint8)                                        #higher kernel = less accurate contours
#close_img = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)            #erosion + dilute method (internal spaces removal)
open_img = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, kernel)               #dilute + erosion method (noise removal)

#Edge detection and countours mapping
canny_img = cv2.Canny(open_img, 50, 100)                                  #edge detection using the OpenCV Canny method
contours, _ = cv2.findContours(open_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)  
selected_contour = 1                                                      #0 = frame; 1 = uno card perimeter;
final_img = cv2.drawContours(rotated_img, contours, selected_contour, (0,255,0), 2, cv2.LINE_AA)


#cv2.drawContours(colour_img, contours, -1, (0, 255, 0), 1)               #draw all countours in green on top of the original colour image
#print(len(contours))                                                     #debugging: show how many contours have been found
#print(hierarchy)

#Contours coordinates manual
#max_coords = max((contours[selected_contour]).tolist())
#contour_x = (max_coords[0])[0]
#contour_y = (max_coords[0])[1]
#print(contours[selected_contour])
#print(max_coords)
#print(contour_x)
#print(contour_y)

#Contour cropped image with coordinates using OpenCV

x_cnt,y_cnt,w_cnt,h_cnt = cv2.boundingRect(contours[selected_contour])
contour_cropped_img = rotated_img[y_cnt:y_cnt+h_cnt, x_cnt:x_cnt+w_cnt] 
framed_img = cv2.copyMakeBorder(contour_cropped_img,30,30,30,30,cv2.BORDER_CONSTANT,value=(0,0,0))
colour_framed_img = cv2.copyMakeBorder(framed_img,5,5,5,5,cv2.BORDER_CONSTANT,value=(243,118,28))
#print(x_cnt, y_cnt, w_cnt, h_cnt)


#Identification text placement
disp_x = +45                                                      #perfect value = +45
disp_y = -75                                                      #perfect value = -75
identified_img = cv2.putText(colour_framed_img,"UNO CARD", (y_cnt+disp_y, x_cnt+disp_x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

#Colour analysis
hsv_pixels = (cv2.cvtColor(contour_cropped_img, cv2.COLOR_BGR2HSV)).tolist()
pixels = [x[0][0] for x in hsv_pixels]
average = sum(pixels)/len(pixels)
frequent_value = max(set(pixels), key = pixels.count)
print(frequent_value)
#print(type(pixels))
print(average)


#Show processed image to user
cv2.imshow('original', colour_img)
cv2.imshow('cropped', contour_cropped_img)
cv2.imshow('final_img',identified_img)
key = cv2.waitKey(0)
cv2.destroyAllWindows()




60
53.65461847389558
